In [1]:
import psycopg2


host = "dumbo.db.elephantsql.com"
database = "shipfodm"
user = "shipfodm"
password = "bJo0TT1z5RGalO3A7zWqLVnSC-mrsYFb"
port = "5432"


conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port=port
)


In [4]:

create_table_query = """
CREATE TABLE Hosts_Table (
    host_id SERIAL PRIMARY KEY,
    First_Name VARCHAR(255),
    Last_Name VARCHAR(255),
    Email VARCHAR(255)
);
"""

with conn.cursor() as cursor:
    cursor.execute(create_table_query)
    conn.commit()

print("Table created successfully!")


In [7]:
create_table_queries = [
    """
    CREATE TABLE Guests_Table (
        guest_id SERIAL PRIMARY KEY,
        First_Name VARCHAR(255),
        Last_Name VARCHAR(255),
        Email VARCHAR(255)
    );
    """,
    """
    CREATE TABLE Rooms_Table (
        room_id SERIAL PRIMARY KEY,
        host INTEGER,
        guest INTEGER,
        number_of_rooms INTEGER,
        max_occupancy INTEGER,
        price INTEGER,
        AC BOOLEAN,
        Refrigerator BOOLEAN,
        FOREIGN KEY (host) REFERENCES Hosts_Table(host_id),
        FOREIGN KEY (guest) REFERENCES Guests_Table(guest_id)
    );
    """,
    """
    CREATE TABLE Reservations_Table (
        reservation_id SERIAL PRIMARY KEY,
        guest INTEGER,
        room INTEGER,
        check_in_date DATE,
        check_out_date DATE,
        total_price INTEGER,
        FOREIGN KEY (guest) REFERENCES Guests_Table(guest_id),
        FOREIGN KEY (room) REFERENCES Rooms_Table(room_id)
    );
    """,
    """
    CREATE TABLE Reviews_Table (
        review_id SERIAL PRIMARY KEY,
        guest INTEGER,
        room INTEGER,
        rating INTEGER,
        comment TEXT,
        FOREIGN KEY (guest) REFERENCES Guests_Table(guest_id),
        FOREIGN KEY (room) REFERENCES Rooms_Table(room_id)
    );
    """
]

conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port=port
)

with conn.cursor() as cursor:
    for query in create_table_queries:
        cursor.execute(query)
    conn.commit()

print("Tables created successfully!")

Tables created successfully!


In [8]:

hosts_data = [
    ("John", "Doe", "john@example.com"),
    ("Jane", "Smith", "jane@example.com"),
    ("Michael", "Johnson", "michael@example.com"),
    ("Emily", "Brown", "emily@example.com"),
    ("David", "Williams", "david@example.com")
]

guests_data = [
    ("Alice", "White", "alice@example.com"),
    ("Bob", "Green", "bob@example.com"),
    ("Linda", "Lee", "linda@example.com"),
    ("James", "Hall", "james@example.com"),
    ("Olivia", "Martin", "olivia@example.com")
]

rooms_data = [
    (1, 2, 2, 4, 150, True, True),
    (2, 3, 1, 3, 120, False, True),
    (1, 1, 1, 2, 100, True, False),
    (3, 4, 3, 5, 200, True, True),
    (2, 5, 2, 4, 150, False, False)
]

reservations_data = [
    (2, 3, '2023-08-01', '2023-08-05', 500),
    (1, 1, '2023-08-10', '2023-08-15', 600),
    (4, 5, '2023-08-05', '2023-08-08', 300),
    (3, 2, '2023-08-20', '2023-08-25', 750),
    (5, 4, '2023-09-01', '2023-09-10', 800)
]

reviews_data = [
    (1, 3, 4, "Great place to stay!"),
    (2, 1, 5, "Not bad, but could be better."),
    (3, 2, 3, "Excellent service and comfortable room."),
    (4, 4, 5, "Average experience."),
    (5, 5, 3, "Highly recommended!")
]

def insert_data(query, data):
    with conn.cursor() as cursor:
        cursor.executemany(query, data)
        conn.commit()

conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port=port
)

insert_hosts_query = "INSERT INTO Hosts_Table (First_Name, Last_Name, Email) VALUES (%s, %s, %s);"
insert_guests_query = "INSERT INTO Guests_Table (First_Name, Last_Name, Email) VALUES (%s, %s, %s);"
insert_rooms_query = "INSERT INTO Rooms_Table (host, guest, number_of_rooms, max_occupancy, price, AC, Refrigerator) VALUES (%s, %s, %s, %s, %s, %s, %s);"
insert_reservations_query = "INSERT INTO Reservations_Table (guest, room, check_in_date, check_out_date, total_price) VALUES (%s, %s, %s, %s, %s);"
insert_reviews_query = "INSERT INTO Reviews_Table (guest, room, rating, comment) VALUES (%s, %s, %s, %s);"

insert_data(insert_hosts_query, hosts_data)
insert_data(insert_guests_query, guests_data)
insert_data(insert_rooms_query, rooms_data)
insert_data(insert_reservations_query, reservations_data)
insert_data(insert_reviews_query, reviews_data)


In [14]:
select_query = "SELECT * FROM Hosts_Table;"

with conn.cursor() as cursor:
    conn.rollback()

    cursor.execute(select_query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)

(1, 'John', 'Doe', None, 'john@example.com')
(2, 'Jane', 'Smith', None, 'jane@example.com')
(3, 'Michael', 'Johnson', None, 'michael@example.com')
(4, 'Emily', 'Brown', None, 'emily@example.com')
(5, 'David', 'Williams', None, 'david@example.com')


In [15]:
find_user_with_most_reservations_query = """
    SELECT g.First_Name, COUNT(r.guest) AS reservation_count
    FROM Guests_Table g
    JOIN Reservations_Table r ON g.guest_id = r.guest
    GROUP BY g.First_Name
    ORDER BY reservation_count DESC
    LIMIT 1;
"""

In [19]:
find_host_with_most_earnings_query = """
    SELECT h.First_Name, SUM(r.total_price) AS earnings
    FROM Hosts_Table h
    JOIN Rooms_Table rt ON h.host_id = rt.host
    JOIN Reservations_Table r ON rt.room_id = r.room
    WHERE DATE_PART('month', r.check_in_date) = DATE_PART('month', CURRENT_DATE - INTERVAL '1 month')
    GROUP BY h.First_Name
    ORDER BY earnings DESC
    LIMIT 1;
"""


In [20]:
find_host_with_best_rating_query = """
    SELECT h.First_Name, AVG(rv.rating) AS average_rating
    FROM Hosts_Table h
    JOIN Rooms_Table rt ON h.host_id = rt.host
    JOIN Reviews_Table rv ON rt.room_id = rv.room
    GROUP BY h.First_Name
    ORDER BY average_rating DESC
    LIMIT 1;
"""

In [23]:
with conn.cursor() as cursor:
    try:
        cursor.execute(find_user_with_most_reservations_query)
        user_with_most_reservations = cursor.fetchone()
        print("User with most reservations:", user_with_most_reservations)

        cursor.execute(find_host_with_most_earnings_query)
        host_with_most_earnings = cursor.fetchone()
        print("Host with most earnings:", host_with_most_earnings)

        cursor.execute(find_host_with_best_rating_query)
        host_with_best_rating = cursor.fetchone()
        print("Host with best rating:", host_with_best_rating)

    except Exception as e:
        print("Error:", e)

User with most reservations: ('Alice', 1)
Host with most earnings: None
Host with best rating: ('Michael', Decimal('5.0000000000000000'))
